In [ ]:
#這是jupyter notebook的magic word˙
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from IPython import display

In [ ]:
import os
#判斷是否在jupyter notebook上
def is_in_ipython():
    "Is the code running in the ipython environment (jupyter including)"
    program_name = os.path.basename(os.getenv('_', ''))

    if ('jupyter-notebook' in program_name or # jupyter-notebook
        'ipython'          in program_name or # ipython
        'jupyter' in program_name or  # jupyter
        'JPY_PARENT_PID'   in os.environ):    # ipython-notebook
        return True
    else:
        return False


#判斷是否在colab上
def is_in_colab():
    if not is_in_ipython(): return False
    try:
        from google import colab
        return True
    except: return False

#判斷是否在kaggke_kernal上
def is_in_kaggle_kernal():
    if 'kaggle' in os.environ['PYTHONPATH']:
        return True
    else:
        return False

if is_in_colab():
    from google.colab import drive
    drive.mount('/content/gdrive')

In [ ]:
os.environ['TRIDENT_BACKEND'] = 'pytorch'

if is_in_kaggle_kernal():
    os.environ['TRIDENT_HOME'] = './trident'
    
elif is_in_colab():
    os.environ['TRIDENT_HOME'] = '/content/gdrive/My Drive/trident'

#為確保安裝最新版 

!pip uninstall tridentx -y
!pip install ../input/trident/tridentx-0.7.3.21-py3-none-any.whl --upgrade
#!pip install cupy

import json
import copy
import numpy as np
#調用trident api
import trident as T
from trident import *

from trident.layers.pytorch_initializers import orthogonal
import random
from tqdm import tqdm
import glob
import scipy
import time

這比賽是基於
0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral

In [ ]:
import pandas as pd
train_df = pd.read_csv('../input/challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv')
train_df

In [ ]:
counts = train_df['emotion'].value_counts().to_dict()
print(counts)

In [ ]:
#train_df


data=open('../input/challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv','r',encoding='utf-8-sig').readlines()
print(len(data))
array2image(np.array(eval('['+data[1].split(',')[-1].replace(' ',', ')+']')).reshape((48,48)))

In [ ]:
emotions=[]
images=[]
test_emotions=[]
test_images=[]
for i in tqdm(range(len(data)-1)):
    row=data[i+1]
    cols=row.split(',')
    if cols[1]=='Training':
        emotions.append(int(cols[0]))
        images.append(np.array(eval('['+cols[-1].replace(' ',', ')+']')).reshape((48,48,1)).astype(np.float32))
    else:
        test_emotions.append(int(cols[0]))
        test_images.append(np.array(eval('['+cols[-1].replace(' ',', ')+']')).reshape((48,48,1)).astype(np.float32))
        

print(len(emotions))
print(len(images))
print(len(test_emotions))
print(len(test_images))






在上一版分析中發現，disgust(1)這種表情因為數量過少，因此決策邊界不明確而成為效度提升的最大問題點，因此在這版中我刻意把disgust案例挑選出來，並且增量5後被放入樣本(只放訓練集，測試集不動)，希望能透過樣本數增加來強化它的決策邊界。

In [ ]:
label_dict=OrderedDict()
test_label_dict=OrderedDict()
disgust_images1=[]
disgust_emotions1=[]
for lab,img in zip(emotions,images):
    if lab not in label_dict:
        label_dict[lab]=1
    else:
        label_dict[lab]+=1
    if lab==1:
        disgust_images1.append(img)
        disgust_emotions1.append(lab)
test_disgust_images1=[]
test_disgust_emotions1=[]        
for lab,img in zip(test_emotions,test_images):
    if lab not in test_label_dict:
        test_label_dict[lab]=1
    else:
        test_label_dict[lab]+=1
    if lab==1:
        test_disgust_images1.append(img)
        test_disgust_emotions1.append(lab)
print(len(disgust_images1))
print(label_dict)
print(test_label_dict)

在數據增強部分，像教於上一個案例，我們加入了仿設轉換，以及CLAHE(限制對比度自適應直方圖均衡化)，後者可以強化圖片的對比均衡，能夠將細微的肌肉線條更為凸顯，以便獲取五官以外的肌肉紋理特徵。

In [ ]:
ds1=ImageDataset(images+5*disgust_images1,symbol='images')
ds2=LabelDataset(emotions+5*disgust_emotions1,symbol='emotions_label')
ds2.class_names=['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

test_ds1=ImageDataset(test_images,symbol='images')
test_ds2=LabelDataset(test_emotions,symbol='emotions_label')
test_ds2.class_names=['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

data_provider=DataProvider(traindata=Iterator(data=ds1,label=ds2),testdata=Iterator(data=test_ds1,label=test_ds2))


data_provider.traindata.data.image_transform_funcs=[
    ToRGB(),
    Resize(output_size=(112,112)),
    CLAHE(),
    RandomTransform(rotation_range=25,zoom_range=0.1, shift_range=0.05,  shear_range= 0.1,random_flip= 0.3),
    RandomAdjustGamma(gamma_range=(0.6,1.4)),
    RandomAdjustContrast(value_range=(0.6,1.4)),
    RandomAdjustHue(value_range=(-0.3,0.3)),#調整色相
    RandomAdjustSaturation(value_range=(0.6,1.4)),#調整飽和度
    RandomBlur(ksize_range=(3,7)),#隨機模糊
    SaltPepperNoise(prob=0.001),#椒鹽噪音
    RandomErasing(size_range=(0.08,0.2),transparency_range=(0.4,0.6),transparancy_ratio=0.8),
    Normalize(127.5,127.5)]


data_provider.testdata.data.image_transform_funcs=[
    ToRGB(),
    Resize(output_size=(112,112)),
    CLAHE(),
    Normalize(127.5,127.5)]

In [ ]:
img_data,emotion_data=data_provider.next()
print(img_data.shape,img_data.mean())
print(emotion_data.shape)
test_img_data,test_emotion_data=data_provider.next_test()
print(test_img_data.shape,test_img_data.mean())
data_provider.preview_images()

這次的骨幹是基於「口罩人臉識別」實作的成果，等於是arcFace再加上理解特徵點幾何位置的能力，以及復原臉部的腦捕能力的骨幹。由於表情與特徵點息息相關，因此使用這個骨幹效果會比一般骨幹來的更好。首先建構與「口罩人臉識別」相同的模型結構，並且載入訓練權重。

In [ ]:
from trident.models import arcfacenet
num_faces=10575
#標準生成結構
#不包含原有分類器
se_resnet50 =arcfacenet.SEResNet_IR_50_512(include_top=False,
             pretrained=True,
             freeze_features=True,
             input_shape=(3,112,112))

se_resnet50.model.trainable=False
se_resnet50.model.add_module('last_conv',Conv2d((1,1),7,strides=1,auto_pad=True,use_bias=False,activation=None))
se_resnet50.model.add_module('pool',GlobalAvgPool2d())
se_resnet50.model.add_module('fc',Dense(7))
se_resnet50.model.add_module('softmax',SoftMax(-1,add_noise=True,noise_intensity=0.08))

se_resnet50.model.pool.keep_output=True

is_resume=False
if is_resume and os.path.exists('./Models/emotions_dtection.pth'):
    se_resnet50.load_model('./Models/emotions_dtection.pth')
    print('./Models/emotions_dtection.pth loaded')
else:
    if os.path.exists('../input/facial-expression/Models/emotions_dtection.pth'):
        se_resnet50.load_model('../input/facial-expression/Models/emotions_dtection.pth')
se_resnet50.summary()

將整個模型設定為不可訓練，移除掉最後一層的head，然後依序加入一層通道數為7的卷積層，再透過GlobalAvgPool2d移除掉空間維度，再加上全連接層以及SoftMax即可完成第一個fintune的模型。

為了能夠有效的比較各種技巧對於模型效果的差異，我在此又建構了第二個finetune model

In [ ]:
#標準生成結構
#不包含原有分類器
se_resnet50_2 =arcfacenet.SEResNet_IR_50_512(include_top=False,
             pretrained=True,
             freeze_features=True,
             input_shape=(3,112,112))

se_resnet50_2.model.trainable=False





is_resume=False
if is_resume and os.path.exists('./Models/emotions_dtection2.pth'):
    se_resnet50_2.load_model('./Models/emotions_dtection2.pth')
    print('./Models/emotions_dtection2.pth loaded')
else:
    if os.path.exists('../input/facial-expression/Models/emotions_dtection.pth'):
        se_resnet50_2.load_model('../input/facial-expression/Models/emotions_dtection.pth')

        
se_resnet50_2.model.add_module('last_conv',Conv2d((1,1),512,strides=1,auto_pad=True,use_bias=False,activation=None))
se_resnet50_2.model.add_module('pool',GlobalAvgPool2d())
se_resnet50_2.model.pool.keep_output=True

classifier=Sequential(
    Dense(7,name='fc'),
    SoftMax(-1,add_noise=True,noise_intensity=0.08,name='softmax'))

head=ModuleDict({'classifier':classifier,
                    'features':Identity()
                    },is_multicasting=True)

se_resnet50_2.model.add_module('head',head)


if is_resume and os.path.exists('./Models/emotions_dtection2.pth'):
    se_resnet50_2.load_model('./Models/emotions_dtection2.pth')
    print('./Models/emotions_dtection2.pth loaded')
    
se_resnet50_2.summary()

其中最大的差異在於加入了類別活化映射模組(Class Activation Mapping, CAM)，基本上這等於是特徵內的注意力機制，我們首先透過一個卷積層將通道數降為7(等於分類類別數)，然後再將輸出複製一份，透過GlobalAvgPool2d將它去重空間維度後，再透過1x1卷積再變為二為空間結構，再與原來的輸出進行點乘後，透過Sigmoid將其數值確保介於0\~1之間，這樣的結構可以強化通道與類別的偶和強度，來獲得更好的分類效果。

透過撰寫以training_context為引數的函數，搭配模型的trigger_when，就可以輕鬆地在學習階段與指定時點，執行指定的任務，我們在與預計在第5,10個epoch開始，開放兩層網路的權重供其訓練。

In [ ]:
def unfreeze(training_context):
    if training_context['steps']==0:
        training_context['current_model'].body[23].trainable=True
        training_context['current_model'].body[22].trainable=True

In [ ]:

from trident.callbacks.lr_schedulers import AdjustLRCallbackBase

class PolyLR(AdjustLRCallbackBase):
    def __init__(self,max_lr=1e-3,  max_iter=10000):
        super().__init__()
        self.max_lr = max_lr
        self.max_iter=max_iter
    def on_batch_end(self, training_context):
        current_step =training_context['steps']
        lr = self.max_lr * (1 - (current_step/ self.max_iter)) * (1 - (current_step / self.max_iter))
        if (lr < 1.0e-7):
            lr = 1.0e-7
        self.adjust_learning_rate(training_context, lr, verbose=False)



In [ ]:


se_resnet50.with_optimizer(optimizer=DiffGrad, lr=5e-4, betas=(0.9, 0.999),gradient_centralization='all') \
    .with_loss(CrossEntropyLoss(label_smooth=True)) \
    .with_metric(accuracy, name='accuracy') \
    .with_regularizer('l2',reg_weight=1e-5) \
    .with_accumulate_grads(10)\
    .with_model_save_path('./Models/emotions_dtection.pth')\
    .trigger_when(when='on_batch_end',frequency=1,unit='batch',action=unfreeze)\
    .with_automatic_mixed_precision_training()\
    .with_learning_rate_scheduler(PolyLR(max_lr=5e-4,max_iter=3000))\
    

在第二個finetune模型中，我還加入了center_loss，它有助於將決策邊界變得更清晰。

In [ ]:

def class_accuracy(classifier,target):
    return accuracy(classifier,target)

center_loss=CenterLoss(num_classes=7, feat_dim=512)
    
se_resnet50_2.with_optimizer(optimizer=AdaBelief, lr=1e-3, betas=(0.9, 0.999),gradient_centralization='all') \
    .with_loss(CrossEntropyLoss(auto_balance=True,input_names=['classifier','target'])) \
    .with_loss(center_loss, loss_weight=0.2) \
    .with_metric(class_accuracy, name='accuracy') \
    .with_regularizer('l2',reg_weight=1e-5) \
    .with_accumulate_grads(10)\
    .with_model_save_path('./Models/emotions_dtection2.pth')\
    .with_learning_rate_scheduler(PolyLR(max_lr=1e-3,max_iter=3000))\
    .trigger_when(when='on_batch_end',frequency=1,unit='batch',action=unfreeze)\


#把center loss的中心點權重納入優化器
se_resnet50_2.optimizer.param_groups[0]['params'].append(center_loss.centers)

    

In [ ]:
#
plan=TrainingPlan()\
    .add_training_item(se_resnet50, name='arcface')\
    .add_training_item(se_resnet50_2, name='arcface2')\
    .with_data_loader(data_provider)\
    .repeat_epochs(15)\
    .with_batch_size(64)\
    .print_progress_scheduling(10,unit='batch')\
    .out_sample_evaluation_scheduling(100,unit='batch')\
    .display_loss_metric_curve_scheduling(200,unit='batch',imshow=True)\
    .save_model_scheduling(50,unit='batch')

In [ ]:
plan.start_now()

In [ ]:
from sklearn import manifold
colors = ['#ff0000', '#ffff00', '#00ff00', '#00ffff', '#0000ff',
         '#ff00ff', '#990000', '#999900', '#009900', '#009999']

emotions_legend=['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

features=[]
features2=[]
emotion_data=[]
for n in range(10):
    img_data2,emotion_data2=data_provider.next()
    se_resnet50.model.eval()
    se_resnet50_2.model.eval()
    out=se_resnet50.model(to_tensor(img_data2))
    out2=se_resnet50_2.model(to_tensor(img_data2))
    emotion_data.append(to_numpy(emotion_data2))
    features.append(to_numpy(se_resnet50.model.pool.output))
    features2.append(to_numpy(se_resnet50_2.model.pool.output))
emotion_data=np.concatenate(emotion_data,axis=0)
features=np.concatenate(features,axis=0)
features2=np.concatenate(features2,axis=0)

tsne = manifold.TSNE(n_components=2, init='random',metric='cosine', random_state=0)  # 利用t-sne將512特徵向量降維至2
features_tsne = tsne.fit_transform(features)
features_tsne=l2_normalize(features_tsne)

tsne2 = manifold.TSNE(n_components=2, init='random',metric='cosine', random_state=0)  # 利用t-sne將512特徵向量降維至2
features_tsne2 = tsne2.fit_transform(features2)
features_tsne2=l2_normalize(features_tsne2)

fig = plt.figure(figsize=(18,8))
    
ax1= fig.add_subplot(1, 2, 1)
for i in range(7):
    x_i = features_tsne[:,0][emotion_data==i]
    y_i = features_tsne[:,1][emotion_data==i]
    ax1.scatter(x_i,y_i,s=100,marker='o',c=colors[i])

ax2= fig.add_subplot(1, 2, 2)
for i in range(7):
    x_i = features_tsne2[:,0][emotion_data==i]
    y_i = features_tsne2[:,1][emotion_data==i]
    ax2.scatter(x_i,y_i,s=100,marker='o',c=colors[i])
    
plt.legend(emotions_legend, loc ='lower left')

    

我們看一下基於前面版本所繪製出來的決策邊界圖如下，很明顯的Disgust出現的比例少而且被淹沒在其他情緒中。這很明顯是個不均衡的分布。

<img src='https://docs.google.com/uc?export=download&id=1dPOpAlV06PwncZtYl_pRKKq_-QYSbqyT'/>